In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu
import pandas as pd
from statsmodels.stats.proportion import proportions_ztest

import matplotlib.pyplot as plt
import seaborn as sns

/Users/mahdie/opt/anaconda3/envs/physioai/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### Physical activity

In [2]:
dfPA_treatment = pd.read_csv("testdata/2.PA_treatment.csv")
dfPA_control = pd.read_csv("testdata/2.PA_control.csv")
dfPA_balanced = pd.read_csv("testdata/3.PA_balanced.csv")

In [3]:
# List of file paths and corresponding dataframe names
files = [
    ("testdata/2.PA_treatment.csv", "dfPA_treatment"),
    ("testdata/2.PA_control.csv", "dfPA_control"),
    ("testdata/3.PA_balanced.csv", "dfPA_balanced"),
]

# Dictionary to store dataframes
dataframes = {}

# Loop through each file and read the csv into a dataframe, then store it in the dictionary
for file, name in files:
    dataframes[name] = pd.read_csv(file)
    print(f"{name}: {dataframes[name].shape}")

dfPA_treatment: (7603, 9)
dfPA_control: (1169, 9)
dfPA_balanced: (1169, 10)


##### Create column outcome in categorical:

In [4]:
# Define the function to categorize PA_changes
def categorize_change(PA_change):
    if PA_change > 0:
        return 'Increased'
    elif PA_change < 0:
        return 'Decreased'
    else:
        return 'Unchanged'

# Apply the function to create a new column in df_PA_control_before
dfPA_treatment['PA_change_category'] = dfPA_treatment['PA_change'].apply(categorize_change)
dfPA_control['PA_change_category'] = dfPA_control['PA_change'].apply(categorize_change)

##### Calculate Chi2:

In [5]:
dfPA_treatment['Group'] = 'Treatment'
dfPA_control['Group'] = 'Control'


df_combined = pd.concat([dfPA_treatment, dfPA_control])

# Create a contingency table for PA_change_category across Treatment and Control groups
contingency_table = pd.crosstab(df_combined['PA_change_category'], df_combined['Group'])

# Perform chi-squared test
chi2, p, dof, ex = chi2_contingency(contingency_table)

print(f"Chi-squared Test for PA_change_category across Treatment and Control groups")
print(f"Chi2: {chi2}, p-value: {p}, Degrees of Freedom: {dof}")
print("\nExpected Frequencies:")
print(ex)
print("\nObserved Frequencies:")
print(contingency_table)


Chi-squared Test for PA_change_category across Treatment and Control groups
Chi2: 24.406773809422873, p-value: 5.013446753257896e-06, Degrees of Freedom: 2

Expected Frequencies:
[[ 335.02804378 2178.97195622]
 [ 471.89113087 3069.10886913]
 [ 362.08082535 2354.91917465]]

Observed Frequencies:
Group               Control  Treatment
PA_change_category                    
Decreased               401       2113
Increased               410       3131
Unchanged               358       2359


#### Quality of life

In [6]:
dfQ_treatment = pd.read_csv("testdata/2.QOL_treatment.csv")
dfQ_control = pd.read_csv("testdata/2.QOL_control.csv")
dfQ_balanced = pd.read_csv("testdata/3.QOL_balanced.csv")

In [7]:
# List of file paths and corresponding dataframe names
files = [
    ("testdata/2.QOL_treatment.csv", "dfQ_treatment"),
    ("testdata/2.QOL_control.csv", "dfQ_control"),
    ("testdata/3.QOL_balanced.csv", "dfQ_balanced"),
]

# Dictionary to store dataframes
dataframes = {}

# Loop through each file and read the csv into a dataframe, then store it in the dictionary
for file, name in files:
    dataframes[name] = pd.read_csv(file)
    print(f"{name}: {dataframes[name].shape}")

dfQ_treatment: (7603, 9)
dfQ_control: (1169, 9)
dfQ_balanced: (1169, 10)


In [8]:
# Define the function to categorize PA_changes
def QOL_categorize_significant_change(QOL_change):
    if QOL_change >= 10:
        return 'Increased 10+ points'
    elif QOL_change <= -10:
        return 'Decreased 10+ points'
    else:
        return 'Maintained'

dfQ_treatment['QOL_change_category'] = dfQ_treatment['QOL_change'].apply(QOL_categorize_significant_change)
dfQ_control['QOL_change_category'] = dfQ_control['QOL_change'].apply(QOL_categorize_significant_change)
dfQ_balanced['QOL_change_category'] = dfQ_balanced['QOL_change'].apply(QOL_categorize_significant_change)

In [9]:
dfQ_control

,Age,Gender,BMI,Depression,Employment_status,Baseline_Physical_Activity,Baseline_Pain,Baseline_Quality_of_life,QOL_change,QOL_change_category
0,75.0,1.0,27.2,1.0,3.0,1,0.0,75.00,25.00,Increased 10+ points
1,77.0,1.0,33.6,1.0,3.0,1,50.0,37.50,12.50,Increased 10+ points
2,76.0,1.0,31.0,2.0,3.0,1,10.0,56.25,6.25,Maintained
3,69.0,1.0,31.8,2.0,1.0,1,30.0,62.50,6.25,Maintained
4,74.0,1.0,29.6,1.0,1.0,1,70.0,50.00,-6.25,Maintained
...,...,...,...,...,...,...,...,...,...,...
1164,46.0,2.0,26.9,2.0,1.0,10,60.0,56.25,12.50,Increased 10+ points
1165,54.0,1.0,27.6,2.0,1.0,10,20.0,68.75,0.00,Maintained
1166,56.0,2.0,27.5,1.0,1.0,10,40.0,43.75,12.50,Increased 10+ points
1167,57.0,1.0,33.6,2.0,1.0,10,40.0,68.75,0.00,Maintained


In [10]:
# Add a new column to identify the group
dfQ_treatment['Group'] = 'Treatment'
dfQ_control['Group'] = 'Control'

# Combine the dataframes
dfQ_combined = pd.concat([dfQ_treatment, dfQ_control])

# Create a contingency table for QOL_change_category across Treatment and Control groups
contingency_table = pd.crosstab(dfQ_combined['QOL_change_category'], dfQ_combined['Group'])

# Perform chi-squared test
chi2, p, dof, ex = chi2_contingency(contingency_table)

# Display results
print(f"Chi-squared Test for QOL_change_category across Treatment and Control groups")
print(f"Chi2: {chi2}, p-value: {p}, Degrees of Freedom: {dof}")
print("\nExpected Frequencies:")
print(ex)
print("\nObserved Frequencies:")
print(contingency_table)


Chi-squared Test for QOL_change_category across Treatment and Control groups
Chi2: 87.26888705870027, p-value: 1.1215069113307754e-19, Degrees of Freedom: 2

Expected Frequencies:
[[ 151.78875969  987.21124031]
 [ 546.51949384 3554.48050616]
 [ 470.69174647 3061.30825353]]

Observed Frequencies:
Group                 Control  Treatment
QOL_change_category                     
Decreased 10+ points      226        913
Increased 10+ points      414       3687
Maintained                529       3003


#### Pain

In [11]:
dfpain_treatment = pd.read_csv("testdata/2.Pain_treatment.csv")
dfpain_control = pd.read_csv("testdata/2.Pain_control.csv")
dfpain_balanced = pd.read_csv("testdata/3.Pain_balanced.csv")

In [12]:
# List of file paths and corresponding dataframe names
files = [
    ("testdata/2.Pain_treatment.csv", "dfpain_treatment"),
    ("testdata/2.Pain_control.csv", "dfpain_control"),
    ("testdata/3.Pain_balanced.csv", "dfpain_balanced")
]

# Dictionary to store dataframes
dataframes = {}

# Loop through each file and read the csv into a dataframe, then store it in the dictionary
for file, name in files:
    dataframes[name] = pd.read_csv(file)
    print(f"{name}: {dataframes[name].shape}")


dfpain_treatment: (7603, 9)
dfpain_control: (1169, 9)
dfpain_balanced: (1169, 10)


In [13]:
# Define the function to categorize changes of at least 20 points
def pain_categorize_significant_change(pain_change):
    if pain_change <= -20:
        return 'Decreased 20+ points'
    elif pain_change >= 20:
        return 'Increased 20+ points'
    else:
        return 'Maintained'  # This now clearly means the pain changed but less than 20 points either way

dfpain_treatment['pain_change_category'] = dfpain_treatment['pain_change'].apply(pain_categorize_significant_change)
dfpain_control['pain_change_category'] = dfpain_control['pain_change'].apply(pain_categorize_significant_change)
dfpain_balanced['pain_change_category'] = dfpain_balanced['pain_change'].apply(pain_categorize_significant_change)


In [14]:
# Add a new column to identify the group
dfpain_treatment['Group'] = 'Treatment'
dfpain_control['Group'] = 'Control'

# Combine the dataframes
dfpain_combined = pd.concat([dfpain_treatment, dfpain_control])

# Create a contingency table for pain_change_category across Treatment and Control groups
contingency_table = pd.crosstab(dfpain_combined['pain_change_category'], dfpain_combined['Group'])

# Perform chi-squared test
chi2, p, dof, ex = chi2_contingency(contingency_table)

# Display results
print(f"Chi-squared Test for pain_change_category across Treatment and Control groups")
print(f"Chi2: {chi2}, p-value: {p}, Degrees of Freedom: {dof}")
print("\nExpected Frequencies:")
print(ex)
print("\nObserved Frequencies:")
print(contingency_table)


Chi-squared Test for pain_change_category across Treatment and Control groups
Chi2: 203.32920636548627, p-value: 7.040829584880354e-45, Degrees of Freedom: 2

Expected Frequencies:
[[ 443.63896489 2885.36103511]
 [ 111.67601459  726.32398541]
 [ 613.68502052 3991.31497948]]

Observed Frequencies:
Group                 Control  Treatment
pain_change_category                    
Decreased 20+ points      337       2992
Increased 20+ points      241        597
Maintained                591       4014
